In [1]:
from tkinter import *
from tkinter import filedialog
from PIL import Image, ImageTk
import math
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from matplotlib.patches import Rectangle
import os

from mpl_toolkits import mplot3d

In [2]:
path="./COMP9517 20T2 Group Project Image Sequences/PhC-C2DL-PSC/Sequence 1/"
class Task3:
    def __init__(self, path):
        self.root=Tk()
        self.frame = Frame(self.root, bd=2, relief=SUNKEN)
        self.frame.grid_rowconfigure(0, weight=1)
        self.frame.grid_columnconfigure(0, weight=1)
        self.label = Label(self.root, text="Click On The Screen")
        self.img_paths=sorted(os.listdir(path))
        self.count=0
        self.path=path
        xscroll = Scrollbar(self.frame, orient=HORIZONTAL)
        xscroll.grid(row=1, column=0, sticky=E+W)
        yscroll = Scrollbar(self.frame)
        yscroll.grid(row=0, column=1, sticky=N+S)
        self.canvas = Canvas(self.frame, bd=0, xscrollcommand=xscroll.set, yscrollcommand=yscroll.set)
        self.image_on_canvas=None
        
#     def detect_cell(self, x, y):
#         maxi=1000
#         label=None
#         for key in trajectory.keys():
#             mid_x, mid_y=trajectory[key][0], trajectory[key][1]
#             distance=math.sqrt((mid_x-x)**2+(mid_y-y)**2)
#             if distance < maxi:
#                 maxi=distance
#                 label=key
#         return key
        
    
#     def speed_cell(self, label):
#         if self.count==0:
#             return 0
#         curr_x, curr_y =trajectory[label][self.count]
#         prev_x, prev_y =trajectory[label][self.count-1]
#         return math.sqrt((curr_x-prev_x)**2+(curr_y-prev_y)**2)
    
#     def total_distance(self, label):
#         if self.count==0:
#             return 0
#         total=0
#         for i in range(1, self.count):
#             curr_x, curr_y=trajectory[label][i]
#             prev_x, prev_y=trajectory[label][i-1]
#             if prev_x==-1 and prev_y==-1:
#                 continue
#             total+=math.sqrt((curr_x-prev_x)**2+(curr_y-prev_y)**2)
            
#         return total
    
#     def net_distance(self, label):
#         for i in range(self.count):
#             start_x, start_y=trajectory[label][i]
#             if start_x!=-1:
#                 break
#         curr_x, curr_y=trajectory[label][self.count]
#         return math.sqrt((curr_x-start_x)**2+(curr_y-start_y)**2)
    
#     def confinement_ratio(self, total, net):
#         return total/net

    def printcoords(self, event):
        print (event.x,event.y)
        self.label['text'] = "x: "+str(event.x)+"y: "+str(event.y)
#         label=detect_cell(event.x, event.y)
#         speed=speed_cell(label)
#         total=total_distance(label)
#         net=net_distance(label)
#         confine=confinement_ratio(total, net)
        
    def check(self):
        img = ImageTk.PhotoImage(Image.open(self.path+self.img_paths[0]))
        Image.open(self.path+self.img_paths[0])
        self.canvas.grid(row=0, column=0, sticky=N+S+E+W)
        self.image_on_canvas=self.canvas.create_image(0,0,image=img,anchor="nw")
        self.canvas.config(scrollregion=self.canvas.bbox(ALL))
        self.canvas.bind("<Button 1>",self.printcoords)
        self.frame.pack(fill=BOTH,expand=1)
        self.root.after(100, self.changeimage)
        self.label.pack()
        self.root.mainloop()
          
    def changeimage(self):
        self.count+=1
        if self.count > len(self.img_paths)-1:
            self.root.quit()
            return
        self.image = ImageTk.PhotoImage(Image.open(self.path+self.img_paths[self.count]))
        self.canvas.itemconfig(self.image_on_canvas, image=self.image)
        self.root.after(100, self.changeimage)

In [3]:
task3=Task3(path)
task3.check()

519 495
279 380
346 189
445 250
187 326
137 169
577 401
578 239
648 322
582 446
351 445
183 469
347 299
574 335
719 576
709 513
689 404
490 412
253 412
180 389
